In [1]:
from util.lexicon_dataloader import LexiconDataLoader
import pandas as pd

In [2]:
LEXICON_PATH = "data/VnEmoLex.xlsx"

In [3]:
# Load the lexicon data
lexicon = LexiconDataLoader(LEXICON_PATH)
lexicon_dict = lexicon.data

Lexicon loaded: 10625 words.


In [18]:
print(lexicon_dict.get('sự nổi tiếng'))

{'english': 'celebrity', 'positive': 1, 'negative': 1, 'anger': 0, 'anticipation': 1, 'disgust': 0, 'fear': 0, 'joy': 1, 'sadness': 0, 'surprise': 1, 'trust': 1, 'sum': 6, 'id': 3}


In [1]:
import pandas as pd
dataset = pd.read_excel("data/dataset_gan_nhan_final.xlsx")